# 🏛️ VGGT: Single Image → Multi-View → 3D Reconstruction

This notebook runs the VGGT 3D reconstruction demo in Google Colab.

**Features:**
- Upload a single image → Generate multi-view images using AI → 3D reconstruction
- Upload video/multiple images → Direct 3D reconstruction

**Before you start:** `Runtime` > `Change runtime type` > Select **GPU** (T4 is fine)


In [1]:
!ls

sample_data


In [2]:
!nvidia-smi


Wed Feb  4 10:42:16 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!ls

sample_data


In [4]:
# Clean up and clone repository (skip LFS files to avoid errors)
!rm -rf /content/demo_vggt_3d /content/sample_data

# Clone with GIT_LFS_SKIP_SMUDGE=1 to skip large example files
!GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/hiendang7613vulcan/demo_vggt_3d /content/demo_vggt_3d

%cd /content/demo_vggt_3d
!ls -la


Cloning into '/content/demo_vggt_3d'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 200 (delta 30), reused 197 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (200/200), 115.76 KiB | 8.27 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/demo_vggt_3d
total 344
drwxr-xr-x 6 root root   4096 Feb  4 10:42 .
drwxr-xr-x 1 root root   4096 Feb  4 10:42 ..
-rw-r--r-- 1 root root  24336 Feb  4 10:42 app.py
-rw-r--r-- 1 root root   3537 Feb  4 10:42 CODE_OF_CONDUCT.md
-rw-r--r-- 1 root root 165602 Feb  4 10:42 colab_vggt_demo.ipynb
-rw-r--r-- 1 root root   1242 Feb  4 10:42 CONTRIBUTING.md
-rw-r--r-- 1 root root  23964 Feb  4 10:42 demo_gradio.py
-rw-r--r-- 1 root root  14984 Feb  4 10:42 demo_viser.py
drwxr-xr-x 9 root root   4096 Feb  4 10:42 examples
drwxr-xr-x 9 root root   4096 Feb  4 10:42 .git
-rw-r--r-- 1 root root   1803 Feb  4 10:42 .gitattributes
-rw-r--r-

In [5]:
!gsutil cp   gs://hien7613storage2/lux2.zip .
!unzip lux2.zip

Copying gs://hien7613storage2/lux2.zip...
/ [1 files][  4.6 MiB/  4.6 MiB]                                                
Operation completed over 1 objects/4.6 MiB.                                      
Archive:  lux2.zip
   creating: multiview_output_20260204_160525_440892/images/
  inflating: multiview_output_20260204_160525_440892/images/lux2_ring1_h0_h0_v30.png  
  inflating: multiview_output_20260204_160525_440892/images/lux2_ring1_h120_h120_v30.png  
  inflating: multiview_output_20260204_160525_440892/images/lux2_ring2_h150_h150_v-30.png  
  inflating: multiview_output_20260204_160525_440892/images/lux2_ring2_h90_h90_v-30.png  
  inflating: multiview_output_20260204_160525_440892/images/lux2_ring2_h30_h30_v-30.png  
  inflating: multiview_output_20260204_160525_440892/images/lux2_ring1_h60_h60_v30.png  
  inflating: multiview_output_20260204_160525_440892/images/lux2_origin_h0_v0.png  


In [6]:
# Install all required dependencies
!pip install \
    gradio==5.17.1 \
    viser==0.2.23 \
    hydra-core==1.3.2 \
    numpy==1.26.3 \
    scipy omegaconf opencv-python einops trimesh matplotlib Pillow onnxruntime tqdm requests \
    fal-client pyyaml

print("\n✓ Dependencies installed successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 27.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 52.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 10.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.4/740.4 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 100.9 MB/s eta 0:00:0000:0100:01


✓ Dependencies installed successfully!


In [ ]:
!pip uninstall -y gradio gradio_client
!pip install "gradio==5.17.1"


In [2]:
%cd /content/demo_vggt_3d


/content/demo_vggt_3d


In [4]:
%cd /content/demo_vggt_3d


/content/demo_vggt_3d


In [ ]:

import runpy
import gradio_client.utils as gcu

# Patch for Gradio boolean schema handling bug (required for gradio 5.17.1)
_orig_get_type = gcu.get_type
def _patched_get_type(schema):
    if isinstance(schema, bool):
        return "boolean"
    return _orig_get_type(schema)
gcu.get_type = _patched_get_type

print("🚀 Launching VGGT Gradio Demo...")
print("📋 After launch, click the public URL to open the interface")
print("-" * 50)

# Launch the demo
runpy.run_path("demo_gradio.py", run_name="__main__")


🚀 Launching VGGT Gradio Demo...
📋 After launch, click the public URL to open the interface
--------------------------------------------------
